## ReAct 에이전트

ReAct = Reason + Act

대형 언어 모델(LLMs)을 활용하여 언어 이해와 상호작용적 의사 결정 과제를 해결하는 새로운 접근법

추론(Reasoning)과 행동(Acting)을 결합하여 다양한 작업을 효과적으로 해결하는 것을 목표로 함.  
CoT(Chain-of-Thought) 프롬프팅에 기반하여 동작하는데, 이는 모델이 단순히 추론만 하는 것이 아니라, 추론 과정에서 얻은 정보를 바탕으로 실시간으로 행동을 결정할 수 있게 한다.

#### ReAct의 핵심 원리

추론과 행동을 상호 보완적으로 결합하여 에이전트가 복잡한 문제를 해결하는 데 있어 높은 유연성과 적응성을 발휘하게 하는 것

### 검색 API 연동

검색 API 는 실시간 정보 검색과 구조화된 결과 반환을 통해 AI 모델이 최신 데이터를 활용할 수 있도록 지원하는 강력한 도구

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain_openai import OpenAI

In [3]:
# 검색 쿼리 함수 구현 및 API 호출

import requests
import os
import json

def search_query(query):
    url = "https://google.serper.dev/search"

    payload = json.dumps({
        "q": query
    })
    headers = {
        'X-API-KEY': os.getenv("SERPER_API_KEY"),
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.text

result = search_query("LLM RAG Agent") # 원하는 쿼리를 입력
print(result)

{"searchParameters":{"q":"LLM RAG Agent","type":"search","engine":"google"},"organic":[{"title":"RAG Architecture + LLM Agent = Better Responses - K2view","link":"https://www.k2view.com/blog/rag-architecture-llm-agent/","snippet":"RAG architectures powered by LLM agents retrieve relevant data from internal and external sources to generate more accurate and contextual responses.","position":1},{"title":"Intro to LLM Agents with Langchain: When RAG is Not Enough","link":"https://medium.com/data-science/intro-to-llm-agents-with-langchain-when-rag-is-not-enough-7d8c08145834","snippet":"This walkthrough through the core elements of the LLM agent architecture will help you design functional bots for the cognitive tasks you aim to automate.","date":"Mar 15, 2024","position":2},{"title":"Agentic RAG - GitHub Pages","link":"https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/","snippet":"In this tutorial we will build a retrieval agent. Retrieval agents are useful when 

##### ReAct 에이전트와 검색 API 통합

In [12]:
# ReAct 에이전트와 구글 검색 API를 연동하여 답변 구성

# Search 에이전트를 활용한 검색 쿼리

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(temperature=0)
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="검색 결과와 함께 유용한 답변을 제공합니다."
    )
]

search = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                          verbose=True,
                          max_iterations=4, 
                          early_stopping_method="generate", 
                          handle_parsing_errors=True)
search.run("발롱도르 2024년도 수상자의 출생 정보는?")



> Entering new AgentExecutor chain...
 I should search for the information about the winner of the Ballon d'Or in 2024.
Action: Intermediate Answer
Action Input: "Ballon d'Or 2024 winner birth information"
Observation: 2024 Ballon d'Or winner Rodri ; 2024 Ballon d'Or winner Rodri · 28 October 2024 · Théâtre du Châtelet, Paris · France Football. Missing: birth | Show results with:birth. Rodri and Aitana Bonmati were named the best players in world football at the 2024 Ballon d'Or awards ceremony in Paris on Monday. 1K. 16K. 204K · Square profile picture · Ballon d'Or · @ballondor. ·. Feb 5. Happy birthday to the one and only. @Cristiano ! Our five-time Ballon d'Or winner ... Missing: birth | Show results with:birth. Harry Kane (England, Bayern Munich) and Kylian Mbappé (France, Paris Saint-Germain / Real Madrid) are the 2024 Gerd Müller Trophy winners. Spanish players won both the Men's and Women's Ballons d'Or in Paris on Monday, as Rodri Hernández and Aitana Bonmatí scooped the main

"The winner of the 2024 Ballon d'Or, Rodri, was born in Spain."

In [13]:
# 복잡한 쿼리를 위한 답변 보완 방법

# tool description 보강
tools = [
        Tool(
            name="Intermediate Answer",
            func=search.run,
            description="도구를 사용하여 현재 금융 데이터, 주식 정보 및 기업 지표에 대한 사실적인 답변을 찾아 답변하세요."
        )
    ]

# 에이전트 내 파라미터 조정 (handle_parsing_errors, max_iterations)
agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.SELF_ASK_WITH_SEARCH,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=3
    )


In [14]:
agent.run("코카콜라의 가장 최근 배당금 지급액은 얼마였나요?")



> Entering new AgentExecutor chain...
Could not parse output:  No.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Intermediate answer: Invalid or incomplete response
So the final answer is: Unable to determine.

> Finished chain.


'Unable to determine.'

댭변 내에서 답을 찾았음에도 불구하고 파싱 에러로 출력 형식이 맞지 않아 OutputParserException 에러가 나는 경우가 있다.  
이 경우에는 기본 설정, 프롬프트 템플릿 조정, 별도 쿼리 사용으로 각 단계별로 점진적으로 구조화된 응답을 추가하여 개선할 수 도 있다.

In [15]:
# 프롬프트 템플릿 조정을 통해 agent_kwargs 를 추가

prompt_template = """다음 형식을 사용하세요:

Question: 답변해야 할 입력 질문
Thought: 무엇을 해야 할지 항상 생각하세요
Action: 취해야 할 행동, 항상 "Intermediate Answer"여야 합니다
Action Input: 행동에 대한 입력
Observation: 행동의 결과
... (이 Thought/Action/Action Input/Observation은 N번 반복될 수 있습니다)
Thought: 이제 최종 답변을 알았습니다
Final Answer: 원래 입력된 질문에 대한 최종 답변

Question: {input}
{agent_scratchpad}
"""

In [16]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_openai import OpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper

# 에이전트 세팅 함수
def setup_self_ask_agent():
    """
    셀프 질문 에이전트 설정 및 반환
    """
    llm = OpenAI(temperature=0)
    search = GoogleSerperAPIWrapper()
    tools = [
        Tool(
            name="Intermediate Answer",
            func=search.run,
            description="도구를 사용하여 현재 금융 데이터, 주식 정보 및 기업 지표에 대한 사실적인 답변을 찾아 한국어로 답변하세요."
        )
    ]

    # 에이전트 내 파라미터 조정 (handle_parsing_errors, max_iterations)
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.SELF_ASK_WITH_SEARCH,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=5
    )

    return agent

# 금융 관련 질문에 대해 정보를 가져오는 함수
def get_financial_info(query: str) -> str:
    agent = setup_self_ask_agent()

    # 구조화된 응답을 유도하기 위해 쿼리 형식 지정
    formatted_query = f""" 다음 정보를 찾습니다:
    {query}
    답변의 형식은 다음과 같이 한국어로 답변하세요:
    '[metric] [value]' 다음에 가능한 경우 출처의 날짜를 입력합니다."""

    try:
        response = agent.run(formatted_query)
        return response
    except Exception as e:
        # 가능한 경우 오류에서 유용한 정보를 추출
        if "Intermediate answer:" in str(e):
            try:
                # 오류 메시지에서 실제 답변 추출
                relevant_info = str(e).split("Intermediate answer:")[-1].split("Follow up:")[0].strip()
                return f"Found information: {relevant_info}"

            except:
                pass
        return f"Error occurred: {str(e)}"


In [17]:
get_financial_info('코카콜라의 가장 최근 배당금 지급액은 얼마였나요?')



> Entering new AgentExecutor chain...
 Yes.
Follow up: When was the most recent dividend payment for Coca-Cola?
Intermediate answer: The Coca-Cola Company's latest ex-dividend date was on June 13, 2025 . The KO stock shareholders received the last dividend payment of $0.51 per share on July 1, 2025 . When is The Coca-Cola Company's next dividend payment date? The Coca-Cola Company's next dividend payment will be on July 1, 2025 .
Follow up: What was the amount of the most recent dividend payment for Coca-Cola?
Intermediate answer: The Coca-Cola Company's latest ex-dividend date was on June 13, 2025 . The KO stock shareholders received the last dividend payment of $0.51 per share on July 1, 2025 .
So the final answer is: 배당금 0.51 달러 [출처: June 13, 2025]

> Finished chain.


'배당금 0.51 달러 [출처: June 13, 2025]'

다음 코드를 통해 에이전트에서 도구를간편하게 로드도 가능하다

In [18]:
from langchain.agents import load_tools
tools = load_tools(["google-serper"])

In [19]:
tools

[GoogleSerperRun(api_wrapper=GoogleSerperAPIWrapper(k=10, gl='us', hl='en', type='search', result_key_for_type={'news': 'news', 'places': 'places', 'images': 'images', 'search': 'organic'}, tbs=None, serper_api_key='6dc6e4ebdbc7235c4465f79cf6b65d7b78aeb6c1', aiosession=None))]

## 에이전트 기반 호출

기본 에이전트를 사용하여 다양한 도구를 호출하고 이를 조합하여 문제를 해결하는 방법을 알아보자.

### 기본 에이전트에서의 도구 호출

기본 에이전트는 AI 시스템의 가장 기초적인 형태로, 정해진 규칙에 따라 도구를 호출하고 결과를 생성하는 역할을 한다.  
따라서 복잡한 작업보다는 단순하고 명확한 작업을 처리하는 데 적합하다.

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Adder 와 Multiplier 라는 2가지 도구를 정의하고, 이를 기본 에이전트에서 호출해보자.

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

/home/haiqv/conda/envs/rag/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
class MultiplierInput(BaseModel):
    a: int = Field(description="첫 번째 숫자")
    b: int = Field(description="두 번째 숫자")


def multiply(a: int, b: int) -> int:
    return a * b


multiplier = StructuredTool.from_function(
    func=multiply,
    name="Multiplier",
    description="두 숫자 곱셈",
    args_schema=MultiplierInput,
    return_direct=False,
)

In [4]:
class AdderInput(BaseModel):
    a: int = Field(description="첫 번째 숫자")
    b: int = Field(description="두 번째 숫자")


def add(a: int, b: int) -> int:
    return a + b


adder = StructuredTool.from_function(
    func=add,
    name="Adder",
    description="두 숫자 덧셈",
    args_schema=AdderInput,
    return_direct=False,
)

In [5]:
tools=[multiplier, adder]

In [6]:
# openai tool agent 생성

from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

system_template = """
당신은 수학 계산을 도와주는 AI 어시스턴트입니다.
사용 가능한 도구들:
- Adder: 두 숫자를 더합니다
- Multiplier: 두 숫자를 곱합니다

각 단계별로 계산 과정을 설명하고, 최종 결과를 명확하게 알려주세요.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(model, tools, prompt)

In [7]:
# run agent

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

query = "13과 28을 더하고 40을 곱하면 어떤 결과가 나올까요?"
response = agent_executor.invoke({"input": query, "chat_history": [] })
result = response['output']




> Entering new AgentExecutor chain...

Invoking: `Adder` with `{'a': 13, 'b': 28}`


41
Invoking: `Multiplier` with `{'a': 40, 'b': 1}`


40
Invoking: `Multiplier` with `{'a': 41, 'b': 40}`


1640먼저, 13과 28을 더했습니다:

\[ 13 + 28 = 41 \]

그 다음, 이 결과인 41에 40을 곱했습니다:

\[ 41 \times 40 = 1640 \]

따라서 최종 결과는 **1640**입니다.

> Finished chain.


In [8]:
query = "100과 20을 더하면 어떤 결과가 나올까요?"
response = agent_executor.invoke({"input": query, "chat_history": [] })
result = response['output']



> Entering new AgentExecutor chain...

Invoking: `Adder` with `{'a': 100, 'b': 20}`


120100과 20을 더하면 120이 나옵니다.

> Finished chain.


### ReAct를 활용한 고급 도구 호출

ReAct 에이전트는 여러 도구를 연속적으로 호출하는 경우, 각 도구의 출력 결과를 바탕으로 동적으로 도구 선택을 할 수 있다.  
이 과정에서 일부 도구의 실패나 오류를 고려하고, 다른 대체 도구를 선택하는 로직을 포함할 수 있다.

In [13]:
import requests
import json
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
import os

In [11]:
# 1. 도구 클래스/함수 정의
class StockTool:
    def run(self, input_data):
        # 입력 데이터를 기반으로 주식 관련 작업 수행 함수 작성
        api_key = os.getenv("rapid_api_key")
        url = "https://seeking-alpha.p.rapidapi.com/symbols/get-ratings"

        querystring = {"symbols":input_data.lower()}

        headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        print(response)

        if response.status_code == 200:
            data = response.json()
            result = json.dumps(data)
        else:
            raise ValueError(f"{input_data}에 대한 데이터가 없습니다.")
        return f"주식데이터 입력: {input_data}\n{result}"

class SearchTool:
    def run(self, input_data):
        # 입력 데이터를 기반으로 검색 작업 수행 함수 작성
        url = "https://google.serper.dev/search"

        payload = json.dumps({
            "q": input_data
        })
        headers = {
            'X-API-KEY': os.getenv("SERPER_API_KEY"),
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)

        return response.text

# 2. 도구 객체 생성
stock_tool = StockTool()
search_tool = SearchTool()

In [14]:
# input 에 따른 도구 지정

def choose_tool_based_on_input(input_data):
    # 입력 데이터에 따라 사용할 도구를 동적으로 선택
    if "주식" in input_data:
        return stock_tool
    elif "검색" in input_data:
        return search_tool
    else:
        raise ValueError("입력에 적합한 도구를 찾을 수 없습니다.")

# 입력에 따라 적합한 도구 선택
def dynamic_tool_call(input_data):
    tool = choose_tool_based_on_input(input_data)
    return tool.run(input_data)

result = dynamic_tool_call("AI 연구 검색")
print(result)

{"searchParameters":{"q":"AI 연구 검색","type":"search","engine":"google"},"organic":[{"title":"Consensus: AI-powered Academic Search Engine","link":"https://consensus.app/home/","snippet":"Search through over 200M research papers across every domain of science & academia. Time-saving AI insights. Gain insight faster with our Pro Analysis ...","position":1},{"title":"[Research Toolbox] 논문 검색을 더 빠르고 정확하게! 연구자를 ... - BRIC","link":"https://www.ibric.org/bric/trend/bio-news.do?mode=view&articleNo=9958804&title=%5BResearch+Toolbox%5D+%EB%85%BC%EB%AC%B8+%EA%B2%80%EC%83%89%EC%9D%84+%EB%8D%94+%EB%B9%A0%EB%A5%B4%EA%B3%A0+%EC%A0%95%ED%99%95%ED%95%98%EA%B2%8C%21+%EC%97%B0%EA%B5%AC%EC%9E%90%EB%A5%BC+%EC%9C%84%ED%95%9C+AI+%EB%85%BC%EB%AC%B8+%EA%B2%80%EC%83%89+%EB%8F%84%EA%B5%AC+%EB%B9%84%EA%B5%90","snippet":"오늘은 연구 과정에서 논문을 검색하는 시간을 줄이고 다량의 최신 논문을 효율적으로 정리하는 데 도움을 주는 AI 도구들을 소개하도록 하겠습니다. 1.","date":"Mar 27, 2025","position":2},{"title":"DBpia - 연구를 돕는 똑똑한 학술콘텐츠 플랫폼","link":"https://www.dbpia.co.kr/","s

#### 복합적인 문제 해결에 적합한 ReAct 전략

ReAct 에이전트는 다양한 도구를 결합하여 복잡한 문제를 해결할 수 있다.  
이 과정에서 성능 향상과 안정승을 위해 여러 전략을 도입할 수 있다.

#### 캐싱 전략

캐싱은 이미 처리한 데이터를 저장해 두고, 동일한 요청이 발생할 때 다시 계산하지 않도록 도와준다.

In [15]:
# 검색 결과 캐싱

from functools import lru_cache

# 검색 결과를 캐싱하는 예시
@lru_cache(maxsize=10)
def cached_search(query):
    return search_tool.run(query)

# 캐싱된 검색 호출
result = cached_search("AI 연구 동향")
print(result)

{"searchParameters":{"q":"AI 연구 동향","type":"search","engine":"google"},"organic":[{"title":"2023년 국내외 인공지능 산업 동향 연구 - SPRi - 소프트웨어정책연구소","link":"https://www.spri.kr/posts/view/23728?code=research&study_type=&board_type=&flg=","snippet":"1. 제 목 : 2023년 국내외 인공지능 산업 동향 연구 · 2. 연구 목적 및 필요성; ㅇ 생성AI 기술이 전세계적 이슈로 부상 · 3. 연구의 구성 및 범위; ㅇ 인공 ...","date":"Apr 29, 2024","position":1},{"title":"[ICLR 2024]Foundation Model 분야 최신 연구 동향 - LG AI연구원","link":"https://www.lgresearch.ai/blog/view?seq=451","snippet":"LG AI연구원 또한 인과추론 기반의 인공지능 모델 연구를 진행하며 이러한 흐름에 동참하고 있습니다. 통계적 모델들은 데이터의 분포가 고정되어 있다고 ...","date":"Jul 5, 2024","position":2},{"title":"2024년 AI/ML 20대 최신 기술 연구 동향 (April 1) - Medium","link":"https://medium.com/@favorable_eminence_oyster_546/2024%EB%85%84-ai-ml-17%EB%8C%80-%EC%B5%9C%EC%8B%A0-%EA%B8%B0%EC%88%A0-%EC%97%B0%EA%B5%AC-%EB%8F%99%ED%96%A5-april-1-788f3cc4f6b2","snippet":"언어 AI 연구의 선두주자 'NeuroScript'에서 차세대 Large Language Model(LLM) 'OmniLingua'를 발표했습니다. 이 모델은 기존 LLM의 한계를 극복하고, ...","date"

#### 에러 복구 전략

API 호출 실패 시 재시도하거나 대체 도구를 사용하는 방법을 통해 안정적인 동작을 보장할 수 있다.

In [16]:
# 도구 호출 에러 시 복구

def robust_tool_call(tool, input_data, retries=3):
    for attempt in range(retries):
        try:
            result = tool.run(input_data)
            return result
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == retries - 1:
                raise
            print("재시도...")

# 도구 호출 예시
result = robust_tool_call(search_tool, "AI 도구의 발전")
print(result)


{"searchParameters":{"q":"AI 도구의 발전","type":"search","engine":"google"},"organic":[{"title":"[All Around AI 1편] AI의 시작과 발전 과정, 미래 전망","link":"https://news.skhynix.co.kr/post/all-around-ai-1","snippet":"AI의 시작은 1950년대로 거슬러 올라간다. 1950년, 영국의 수학자 앨런 튜링(Alan Turing)은 기계는 생각할 수 있다고 주장하며, 이를 테스트하기 위한 ...","date":"Mar 15, 2024","position":1},{"title":"인공지능(AI)은 어떻게 발달해왔는가, 인공지능의 역사 - NVIDIA 블로그","link":"https://blogs.nvidia.co.kr/blog/history_of_ai/","snippet":"마치 인간처럼 생각하고 문제를 풀 수 있는 인공지능을 구현하려는 연구는 1970년대까지 활발히 진행되다가, 단순히 간단한 문제 풀이 뿐만 아니라 좀더 ...","date":"Mar 13, 2016","position":2},{"title":"인공지능 - 나무위키","link":"https://namu.wiki/w/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5","snippet":"2020년대에 들어서는 컴퓨터의 계산 능력이 무서울 정도로 발전하고 있고, 그에 따라 쏟아지는 데이터의 양과 종류도 많아지고 있어 비정형 데이터[8]를 처리하는 능력 ...","position":3},{"title":"인공지능은 어떻게 발달해왔는가, 인공지능의 역사 | 인사이트리포트","link":"https://www.samsungsds.com/kr/insights/091517_cx_cvp3.html","snippet":"딥러닝의 등장 이후 인공지능은 빠른 발전을 보이고 있습니다만 2018년 어떤 기술이 화두가 될지 전혀 예측할 수가 없을 정도로 급격하게 변화

#### 멀티스텝 워크플로와 분기 로직

ReAct 에이전트는 여러 도구를 순차적으로 사용하여 복잡한 문제를 해결할 수 있다.

특히, 특정 조건을 만족할 때만 다음 단계로 넘어가는 분기(branching) 로직을 도입하면 복잡한 의사결정 트리를 구현할 수 있다.

이를 통해 더 복잡한 시나리오에서 에이전트가 효과적으로 동작할 수 있도록 할 수 있다.

In [17]:
def multi_step_workflow(input_data):
    # 첫 번째 단계: 웹 검색
    search_result = search_tool.run(input_data)

    # 두 번째 단계: 특정 조건을 만족하면 주식 가격 확인
    if "stock" in input_data:
        stock_price = stock_tool.run(input_data.split()[-1])
        return f"Search Result: {search_result}, Stock Price: {stock_price}"
    else:
        return f"Search Result: {search_result}"

# 워크플로우 실행 예시
result = multi_step_workflow("AAPL 주식 가격은 얼마입니까?")
print(result)

Search Result: {"searchParameters":{"q":"AAPL 주식 가격은 얼마입니까?","type":"search","engine":"google"},"answerBox":{"snippet":"AAPL의 현재 값은 211.43 USD 입니다 - 지난 24시간 사이 −0.37% 내렸습니다. 애플 주식회사의 주가 성능을 더 자세히 살펴보려면 차트를 확인하세요.","snippetHighlighted":["211.43 USD"],"title":"NASDAQ:AAPL 주가 - 애플 주식회사 - 트레이딩뷰","link":"https://kr.tradingview.com/symbols/NASDAQ-AAPL/"},"knowledgeGraph":{"title":"Apple (애플)","type":"Technology company","imageUrl":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT5ITHsQzdzkkFWKinRe1Y4FUbC_Vy3R_M&s=0","description":"Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services.","descriptionSource":"Wikipedia","descriptionLink":"https://en.wikipedia.org/wiki/Apple_Inc.","attributes":{"CEO":"Tim Cook (Aug 24, 2011–)","Customer service":"1 (800) 275-2273","Founded":"April 1, 1976, Los Altos, CA","Headquarters":"Cupertino, CA","Sales

### 에이전트를 활용한 금융 데이터 실적 프로젝트

에이전트 기술을 활용하여 금융 데이터 분석을 실제로 수행하는 방법을 알아보자.

ReAct 및 RAG 에이전트 개념을 금융 분야에 적용하여 주식 시장의 데이터를 실시간으로 수집하고, 이를 분석에 활용하게 구현할 수 있다.

이번 섹션에서는 RAG 에이전트의 단계적 활용과 함께 코드로 설명

한 가지 유의할 점으로는 ReAct 에이전트가 API를 호출해도 LLM이 결국 결과룰 문장으로 정리할 때 잘못된 추론을 삽입할 수 있으니, LLM이 잘못된 결론을 제시하지 않도록 결과를 검증해야 한다.

#### 1. ReAct 에이전트를 사용한 금융 데이터 수집 및 분석

데코레이터 @tool() 사용 설명
- 등록: @tool() 데코레이터를 사용하여 함수를 ReAct 에이전트의 도구 목록에 등록
- 호출 가능: 등록된 함수는 에이전트 실행 시 호출할 수 있는 도구로 인식
- 자동화: 데이터 수집, 처리 등 반복적인 작업을 자동화하는 데 유용하다.

In [ ]:
# 함수 tool 지정 예시

@tool()
def get_company_profile(stock: str) -> str:
    """
    주식 정보를 입력 받아 기업 정보를 반환합니다.
       변수: stock (str): 검색하고자 하는 주식 정보
       결과: JSON 포맷의 회사 프로필 정보
    """

In [19]:
import requests
import json
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel, Field, validator
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from datetime import datetime

In [20]:
# add pydantic
class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for")

    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ValueError('Stock ticker should only contain up to 4 alphabetic characters')
        return v

In [21]:
# add args_schema=SearchInput inside of the tool decorator
@tool(args_schema=SearchInput)
def get_company_profile(stock:str) -> str:
    # 회사명, 섹터명, 기본 이름, 주식의 직원 수와 같은 세부 프로필 가져오기
    """
    Get the company profile and ratings for a given stock ticker.

    Args:
        stock (str): The stock ticker symbol to search for.

    Returns:
        str: The company profile data in JSON format.
    """
    api_key = os.getenv("rapid_api_key")
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-ratings"


    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")

    return result

In [22]:
get_company_profile('AAPL') # Apple Inc.

/tmp/ipykernel_145970/832491152.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_company_profile('AAPL')


'{"data": [{"id": "146,2025-05-12", "type": "rating", "attributes": {"asDate": "2025-05-12", "tickerId": 146, "ratings": {"authorsCount": 40.0, "authorsRating": 2.875, "authorsRatingBuyCount": 9.0, "authorsRatingBuyCount30Day": 9.0, "authorsRatingBuyCount90Day": 21.0, "authorsRatingHoldCount": 15.0, "authorsRatingHoldCount30Day": 15.0, "authorsRatingHoldCount90Day": 29.0, "authorsRatingSellCount": 10.0, "authorsRatingSellCount30Day": 10.0, "authorsRatingSellCount90Day": 21.0, "authorsRatingStrongBuyCount": 2.0, "authorsRatingStrongBuyCount30Day": 2.0, "authorsRatingStrongBuyCount90Day": 3.0, "authorsRatingStrongSellCount": 4.0, "authorsRatingStrongSellCount30Day": 4.0, "authorsRatingStrongSellCount90Day": 6.0, "quantRating": 3.341780376868096, "sellSideRating": 3.91304, "divConsistencyCategoryGrade": 5, "divGrowthCategoryGrade": 1, "divSafetyCategoryGrade": 2, "dividendYieldGrade": 12, "dpsRevisionsGrade": 10, "epsRevisionsGrade": 9, "growthGrade": 11, "momentumGrade": 7, "profitabilit

In [24]:
get_company_profile('ssea')

'{"errors": [{"code": "404", "detail": "Record Not Found", "status": "404", "title": "Record not found"}]}'

In [25]:
# 특정 주식의 경쟁사 정보 호출

@tool(args_schema=SearchInput)
def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = os.getenv("rapid_api_key")
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")

    return result

In [26]:
get_competitors('AAPL')

'{"data": [{"id": "584719", "type": "ticker", "attributes": {"slug": "xiacy", "name": "XIACY", "companyName": "Xiaomi Corporation", "equityType": "stocks", "indexGroup": null, "currency": "USD", "tradingViewSlug": "OTC:XIACY", "exchange": "Pink Current Info", "exchangeDescription": null, "company": "Xiaomi Corporation", "isBdc": false, "visible": true, "searchable": true, "private": false, "pending": false, "isDefunct": false, "followersCount": 4761, "fundTypeId": 8, "articleRtaCount": 1, "newsRtaCount": 3, "divYieldType": null, "mergedOn": null, "primaryEpsConsensusMeanType": "normalized", "isReit": false, "tickerType": "primary"}, "relationships": {"sector": {"data": {"id": "45", "type": "sector"}}, "subIndustry": {"data": {"id": "45202030", "type": "subIndustry"}}}, "meta": {"companyLogoUrlLight": "https://static.seekingalpha.com/cdn/s3/company_logos/mark_vector_light/XIACY.svg", "companyLogoUrlDark": "https://static.seekingalpha.com/cdn/s3/company_logos/mark_vector_dark/XIACY.svg"}

In [27]:
# 에이전트 생성

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile, get_competitors]
llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [28]:
query = "투자를 하려고 하는데 AAPL 이 뭐야?"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: 투자를 하려고 하는데 AAPL 이 뭐야?


> Entering new AgentExecutor chain...
AAPL은 Apple Inc.의 주식 티커(symbol)입니다. Apple Inc.는 미국의 다국적 기술 회사로, 주로 iPhone, iPad, Mac 컴퓨터, Apple Watch, 그리고 다양한 소프트웨어 및 서비스로 잘 알려져 있습니다. AAPL은 나스닥(NASDAQ) 증권 거래소에 상장되어 있으며, 많은 투자자들이 관심을 갖고 있는 주요 기술 주식 중 하나입니다. Apple의 회사 프로필이나 경쟁사에 대한 정보가 필요하시면 말씀해 주세요!

> Finished chain.
Answer: AAPL은 Apple Inc.의 주식 티커(symbol)입니다. Apple Inc.는 미국의 다국적 기술 회사로, 주로 iPhone, iPad, Mac 컴퓨터, Apple Watch, 그리고 다양한 소프트웨어 및 서비스로 잘 알려져 있습니다. AAPL은 나스닥(NASDAQ) 증권 거래소에 상장되어 있으며, 많은 투자자들이 관심을 갖고 있는 주요 기술 주식 중 하나입니다. Apple의 회사 프로필이나 경쟁사에 대한 정보가 필요하시면 말씀해 주세요!


In [29]:
query = "투자를 하려고 하는데 AAPL 이 뭐고 그 경쟁사은 어디야?"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: 투자를 하려고 하는데 AAPL 이 뭐고 그 경쟁사은 어디야?


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "146,2025-05-12", "type": "rating", "attributes": {"asDate": "2025-05-12", "tickerId": 146, "ratings": {"authorsCount": 40.0, "authorsRating": 2.875, "authorsRatingBuyCount": 9.0, "authorsRatingBuyCount30Day": 9.0, "authorsRatingBuyCount90Day": 21.0, "authorsRatingHoldCount": 15.0, "authorsRatingHoldCount30Day": 15.0, "authorsRatingHoldCount90Day": 29.0, "authorsRatingSellCount": 10.0, "authorsRatingSellCount30Day": 10.0, "authorsRatingSellCount90Day": 21.0, "authorsRatingStrongBuyCount": 2.0, "authorsRatingStrongBuyCount30Day": 2.0, "authorsRatingStrongBuyCount90Day": 3.0, "authorsRatingStrongSellCount": 4.0, "authorsRatingStrongSellCount30Day": 4.0, "authorsRatingStrongSellCount90Day": 6.0, "quantRating": 3.341780376868096, "sellSideRating": 3.91304, "divConsistencyCategoryGrade": 5, "divGrowthCategoryGrade": 1, "divSafetyCat

#### 2. 검색 API를 통한 실시간 금융 시장 분석

In [30]:
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime
from langchain_core.tools import StructuredTool
from langchain_core.tools import ToolException

In [31]:
# change ValueError to ToolException
class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for")

    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ToolException('Stock ticker should only contain up to 4 alphabetic characters')
        return v

In [32]:
# 기업 정보 검색 도구

def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = os.getenv("rapid_api_key")
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")

    return result

In [33]:
get_company_profile('AAPL')

'{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening

In [34]:
# 경쟁사 조회 도구
def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = os.getenv("rapid_api_key")
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")

    return result

In [35]:
# 에이전트 생성

get_company_profile_tool = StructuredTool.from_function(
    func=get_company_profile,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

get_competitors_tool = StructuredTool.from_function(
    func=get_competitors,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile_tool, get_competitors_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [36]:
query = "투자를 하려고 하는데 AAPL 이 뭐야?"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: 투자를 하려고 하는데 AAPL 이 뭐야?


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arca

In [37]:
# 실시간 주식 정보 조회 API 테스트

import requests

url = "https://seeking-alpha.p.rapidapi.com/v2/auto-complete"

querystring = {"query":"apple","type":"people,symbols,pages","size":"5"}

headers = {
	"x-rapidapi-key": "58c32f5dd8mshc7da181b67a6bcep110a60jsnbea62f8b288b",
	"x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'people': [{'id': 51516124, 'type': 'user', 'score': 932.25256, 'url': '/user/51516124', 'name': 'Tim <b>Apple</b>', 'slug': 'tim-apple'}, {'id': 30855475, 'type': 'user', 'score': 561.6426, 'url': '/user/30855475', 'name': '<b>Apple</b> Eye', 'slug': 'apple-eye'}, {'id': 51054234, 'type': 'user', 'score': 508.1606, 'url': '/user/51054234', 'name': 'pine-<b>apple</b>', 'slug': 'pine-apple'}, {'id': 107710, 'type': 'author', 'score': 183.82385, 'url': '/author/appleseed-fund', 'name': '<b>Appleseed</b> Fund', 'slug': 'appleseed-fund'}, {'id': 102870, 'type': 'author', 'score': 108.808876, 'url': '/author/victor-dergunov', 'name': 'Victor Dergunov', 'slug': 'victor-dergunov'}], 'symbols': [{'id': 146, 'type': 'symbol', 'score': 32103.209, 'url': '/symbol/AAPL', 'name': 'AAPL', 'content': '<b>Apple</b> Inc.', 'slug': 'aapl'}, {'id': 513836, 'type': 'symbol', 'score': 21140.443, 'url': '/symbol/APLE', 'name': 'APLE', 'content': '<b>Apple</b> Hospitality REIT, Inc.', 'slug': 'aple'}, {'id'

In [38]:
# 주식 정보 조회 도구 
def get_stock_symbol(query: str) -> str:
    url = "https://seeking-alpha.p.rapidapi.com/v2/auto-complete"
    querystring = {"query":query,"type":"people,symbols,pages","size":"5"}
    # 기업명 기반으로 주식 정보를 가져오기 위한 함수
    headers = {
        "x-rapidapi-key": os.getenv("rapid_api_key"),
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        print(data)
        # 첫 번째 결과의 심볼 반환
        if "symbols" in data and len(data["symbols"]) > 0:
            symbol = data['symbols'][0]['name']
            s_id = data['symbols'][0]['id']
            return symbol, s_id
        else:
            return None
    else:
        print(response)
        raise Exception(f"Failed to fetch stock symbol: {response.status_code}")

In [39]:
get_stock_symbol("Apple")

{'people': [{'id': 51516124, 'type': 'user', 'score': 942.1623, 'url': '/user/51516124', 'name': 'Tim <b>Apple</b>', 'slug': 'tim-apple'}, {'id': 30855475, 'type': 'user', 'score': 567.6128, 'url': '/user/30855475', 'name': '<b>Apple</b> Eye', 'slug': 'apple-eye'}, {'id': 51054234, 'type': 'user', 'score': 513.5437, 'url': '/user/51054234', 'name': 'pine-<b>apple</b>', 'slug': 'pine-apple'}, {'id': 107710, 'type': 'author', 'score': 183.37286, 'url': '/author/appleseed-fund', 'name': '<b>Appleseed</b> Fund', 'slug': 'appleseed-fund'}, {'id': 102870, 'type': 'author', 'score': 108.37248, 'url': '/author/victor-dergunov', 'name': 'Victor Dergunov', 'slug': 'victor-dergunov'}], 'symbols': [{'id': 146, 'type': 'symbol', 'score': 32103.209, 'url': '/symbol/AAPL', 'name': 'AAPL', 'content': '<b>Apple</b> Inc.', 'slug': 'aapl'}, {'id': 513836, 'type': 'symbol', 'score': 21140.443, 'url': '/symbol/APLE', 'name': 'APLE', 'content': '<b>Apple</b> Hospitality REIT, Inc.', 'slug': 'aple'}, {'id': 

('AAPL', 146)

In [40]:
# 실시간 주식 정보 조회
def get_realtime_stock_price(symbol: str) -> dict:
    url = "https://seeking-alpha.p.rapidapi.com/market/get-realtime-quotes"
    querystring = {"sa_ids": symbol}
    '''회사의 최신 주가 확인하기'''
    headers = {
        "x-rapidapi-key": os.getenv("rapid_api_key"),
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        return data.get("real_time_quotes", {})
    else:
        raise Exception(f"실시간 데이터 가져오기에 실패했습니다: {response.status_code}")

In [41]:
get_realtime_stock_price('146')

[{'ticker_id': 146,
  'sa_id': 146,
  'sa_slug': 'aapl',
  'symbol': 'AAPL',
  'high': 212.57,
  'low': 209.77,
  'open': 212.36,
  'close': 211.26,
  'prev_close': 211.45,
  'last': 211.26,
  'volume': 54737850,
  'last_time': '2025-05-16T16:00:00.000-04:00',
  'ext_time': '2025-05-16T19:59:55.000-04:00',
  'ext_price': 207.93,
  'ext_market': 'post',
  'info': 'Market Close',
  'src': 'UpdateLastQuote',
  'updated_at': '2025-05-16T21:20:37.274-04:00'}]

In [42]:
# 실시간 주식 정보를 조회하는 fetch 함수
def fetch_stock_price(query: str) -> str:
    try:
        # 1: 주식 심볼 추출
        symbol,s_id = get_stock_symbol(query)
        print(symbol,s_id)
        if not symbol:
            return f"'{query}'에 대한 심볼을 찾을 수 없습니다."

        # 2: 실시간 주식 가격 추출
        stock_data = get_realtime_stock_price(s_id)
        if not stock_data:
            return f"실시간 데이터를 사용할 수 없습니다: '{symbol}'."

        # 관련 값 추출
        price = stock_data[0].get("last", "N/A")
        return f"{symbol}의 현재 가격은 ${price}입니다."
    except Exception as e:
        return str(e)

In [43]:
fetch_stock_price("Apple Inc.")

{'people': [{'id': 386, 'type': 'author_research', 'score': 5691.061, 'url': '/mp/386-yield-hunting-alt-inc-opps', 'name': 'Yield Hunting: Alt <b>Inc</b> Opps - Alpha Gen Capital', 'slug': None}, {'id': 1363, 'type': 'author_research', 'score': 2807.6462, 'url': '/mp/1363-conservative-income-portfolio', 'name': 'Conservative <b>Income</b> Portfolio - Trapping Value', 'slug': None}, {'id': 1189, 'type': 'author_research', 'score': 2765.1072, 'url': '/mp/1189-high-income-diy-portfolios', 'name': 'High <b>Income</b> DIY Portfolios - Financially Free Investor', 'slug': None}, {'id': 1336, 'type': 'author_research', 'score': 2655.477, 'url': '/mp/1336-portfolio-income-solutions', 'name': 'Portfolio <b>Income</b> Solutions - Dane Bowler', 'slug': None}, {'id': 1356, 'type': 'author_research', 'score': 2578.475, 'url': '/mp/1356-inside-the-income-factory', 'name': 'Inside the <b>Income</b> Factory - Steven Bavaria', 'slug': None}], 'symbols': [{'id': 146, 'type': 'symbol', 'score': 220776.1, 

'AAPL의 현재 가격은 $211.26입니다.'

실시간 주가 모니터링 시스템 구현

특정 주식의 실시간 가격 변동을 모니터링하고, 이를 바탕으로 매수/매도 결정을 내릴 수 있도록 검색 API를 통해 특정 주식의 실시간 가격 데이터를 가져와서 설정된 가격 임계값에 도달했을 때 알림을 발생시킬 수 있다.

In [44]:
# 주식 가격 모니터링
def monitor_stock_price(stock: str, threshold: float):
    price_info = fetch_stock_price(stock)
    current_price = float(price_info.split('$')[-1].split('.')[0])

    if current_price >= threshold:
        print(f"{stock} 주가가 ${threshold} 이상입니다. 현재 주가: ${current_price}. 매도 고려하세요.")
    else:
        print(f"{stock} 주가가 ${threshold} 이하입니다. 현재 주가: ${current_price}. 매수 고려하세요.")

In [45]:
# Example Usage
monitor_stock_price("AAPL", 200.00)

{'people': [{'id': 43904656, 'type': 'user', 'score': 2189.4097, 'url': '/user/43904656', 'name': 'All*<b>AAPL</b>', 'slug': 'all-aapl'}, {'id': 83926, 'type': 'author', 'score': 69.70445, 'url': '/author/mark-kusnir', 'name': 'Mark Kusnir', 'slug': 'mark-kusnir'}, {'id': 104905, 'type': 'author', 'score': 66.9439, 'url': '/author/niki-schranz', 'name': 'Niki Schranz', 'slug': 'niki-schranz'}, {'id': 51312, 'type': 'author', 'score': 23.754364, 'url': '/author/jim-wallingford', 'name': 'Jim Wallingford', 'slug': 'jim-wallingford'}, {'id': 98186, 'type': 'author', 'score': 20.279146, 'url': '/author/dividend-nut', 'name': 'Dividend Nut', 'slug': 'dividend-nut'}], 'symbols': [{'id': 146, 'type': 'symbol', 'score': 827402.94, 'url': '/symbol/AAPL', 'name': '<b>AAPL</b>', 'content': 'Apple Inc.', 'slug': 'aapl'}, {'id': 764528, 'type': 'symbol', 'score': 18794.201, 'url': '/symbol/AAPL:CA', 'name': '<b>AAPL:CA</b>', 'content': 'Apple Inc.', 'slug': 'aapl:ca'}, {'id': 719409, 'type': 'symbo

#### 3. ReAct 에이전트를 통한 통합 분석

통합 분석 프로세스는 실시간 데이터를 수집하고 분석하여 투자 전략을 도출하는 전체 절차를 말한다.

최신 뉴스와 주가 변동성을 분석하고, 이를 바탕으로 투자 전략을 수립하는 방법을 구현해보자.

In [46]:
from dotenv import load_dotenv
load_dotenv()

True

In [73]:
from typing import List, Dict, Any
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
import requests
import json
from datetime import datetime, timedelta

In [51]:
import json
from typing import Dict, Any
from datetime import timedelta

In [54]:
# 뉴스 기사 감성 분석
@tool
def get_news_sentiment(topic: str) -> Dict[str, Any]:
    """특정 주제에 대한 뉴스 기사의 감성을 분석합니다."""

    # NewsAPI 설정
    NEWS_API_KEY = os.getenv('NEWS_API_KEY')
    news_api_url = "https://newsapi.org/v2/everything"

    # 날짜 범위 설정
    end_date = datetime.now()
    start_date = end_date - timedelta(days=7)

    querystring = {
        "q": topic,
        "from": start_date.strftime("%Y-%m-%d"),
        "to": end_date.strftime("%Y-%m-%d"),
        "language": "ko",
        "sortBy": "relevancy",
        "apiKey": NEWS_API_KEY
    }

    try:
        response = requests.get(news_api_url, params=querystring)
        articles = response.json().get('articles', [])

        if not articles:
            return {
                "topic": topic,
                "average_sentiment": 0,
                "analyzed_articles": 0,
                "sentiment_summary": "데이터 없음"
            }

        # 감성 분석을 위한 OpenAI 설정
        llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0,
        )

        # 각 기사에 대한 감성 분석
        sentiment_scores = []
        print(articles[:10])
        for article in articles[:10]:  # 최근 10개 기사만 분석
            if article['description'] != '[Removed]':
                prompt = ChatPromptTemplate.from_messages([
                    ("system", "뉴스 기사의 감성을 분석하여 아래와 같은 JSON 형태로 응답하세요. "
                "결과는 'score'는 -1(매우 부정적)에서 1(매우 긍정적) 사이의 값이고, "
                "'explanation'은 해당 감성 점수에 대한 설명을 포함합니다."),
                    ("user", f"제목: {article['title']}\n내용: {article['description']}")
                ])

                chain = prompt | llm
                result = chain.invoke({})
                try:
                    response_json = json.loads(result.content)
                    score = float(response_json['score'])
                    if score != 0:
                        print(response_json)
                    sentiment_scores.append(score)
                except (json.JSONDecodeError, KeyError, ValueError) as e:
                    print(f"파싱 에러 결과: {e}")
                    continue

        # 종합 감성 점수 계산
        if sentiment_scores:
            avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
            return {
                "topic": topic,
                "average_sentiment": round(avg_sentiment, 2),
                "analyzed_articles": len(sentiment_scores),
                "sentiment_summary": "긍정적" if avg_sentiment > 0.3 else "부정적" if avg_sentiment < -0.3 else "중립적"
            }
        else:
            return {
                "topic": topic,
                "average_sentiment": 0,
                "analyzed_articles": 0,
                "sentiment_summary": "분석 불가"
            }

    except Exception as e:
        print('❌',e)
        return {
            "topic": topic,
            "average_sentiment": 0,
            "analyzed_articles": 0,
            "sentiment_summary": f"에러 발생: {str(e)}"
        }

In [55]:
get_news_sentiment('한화')

[{'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '이두리 기자 redo@kyunghyang.com', 'title': '12회 연속 ‘히트’…한화 드라마 ‘대박’', 'description': '와이스, 8이닝 무실점 ‘철벽’\n키움 대파…연승 기록 추가\n“5명 선발 투수들 서로 지원”\n전신 빙그레 14연승에 ‘성큼’\n\n한화 라이언 와이스가 11일 고척 스카이돔에서 열린 키움전에서 깔끔하게 이닝을 마친 뒤 미소 지으며 마운드를 내려오고 있다. 연합뉴스\n\n한화가 지는 법을 잊었다. 벌써 12연승이다. 구단 최다 연승 기록인 14연승까지 2···', 'url': 'https://www.khan.co.kr/article/202505112020015', 'urlToImage': 'https://img.khan.co.kr/news/2025/05/11/l_2025051201000278400025171.jpg', 'publishedAt': '2025-05-11T11:20:01Z', 'content': '12 \r\n11 . \r\n. 12. 14 2 .\r\n 11 8-0 . 33 11 .\r\n 12 1992 523 . 2 14 . .\r\n . 1~2 4 3 4 .\r\n 3 . 5-0 9 , 3 .\r\n 6 . 8 93 1 2, 9 .\r\n 8 , , , 93 1 .\r\n . 5 .\r\n 6 1 . 16 55 3.73 8 6 .\r\n .\r\n . 1 .\r\n . 11 . 2 2 .… [+21 chars]'}, {'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '이성희 기자 mong2@kyunghyang.com', 'title': '‘새우가 고래 삼켰다’ 한화, 아워홈 인수···단체급식 판도 바뀌나', 'description': '‘한화 삼남’ 김동선 부사장 주도\n업계선 ‘범LG가 물량’ 어디로 갈까 관심\n구지은 부회장 향후 반발 등 과

{'topic': '한화',
 'average_sentiment': 0.48,
 'analyzed_articles': 10,
 'sentiment_summary': '긍정적'}

In [67]:
# 주요 경제 지표 관련 뉴스기사 감성 분석 도구
@tool
def analyze_economic_indicators(indicators: Dict[str, Any] = None) -> Dict[str, Any]:
    """주요 경제 지표 관련 뉴스를 분석합니다."""
    default_indicators = ["금리", "통화정책", "인플레이션", "GDP"]
    if indicators is None:
        indicators ={"list": default_indicators}
    economic_indicators = indicators.get('list', default_indicators)

    results = {}
    for indicator in economic_indicators:
        results[indicator] = get_news_sentiment(indicator)
    return results

In [70]:
analyze_economic_indicators({})

[{'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '최미랑 기자 rang@kyunghyang.com', 'title': '건설경기 ‘최악’…금융위기 때보다 더 심각', 'description': '건산연 ‘진단 보고서’…“더 빠르게 침체, 더 느리게 회복” 전망\n\n\n\n수주·착공 감소에 수익성도 하락 \n저성장 국면 진입, 반등 여건 악화\n\n“금리 인하·재정 지출 확대 제약 \n정부, 신속한 ‘부양책’ 마련 필요” \n공공 발주 정상화·도심 개발 제안\n\n국내 건설경기가 2008년 글로벌 금융위기 때보다 더 침체된 수준이며 경기 여건상 회···', 'url': 'https://www.khan.co.kr/article/202505152050005', 'urlToImage': 'https://img.khan.co.kr/news/2025/05/15/l_2025051601000413500042541.jpg', 'publishedAt': '2025-05-15T11:50:00Z', 'content': ', \r\n· , \r\n · , · \r\n 2008 .\r\n() 15 2008 2008 . , , , .\r\n 2023 2008 . .\r\n () 2023 2071000, (2484000) 16.6% . 2008(-6.1%) .\r\n , 2023 () 1719000 18.9% . 2008 15.5% .\r\n () 3.9% , 1785000 2007(1842000) . 2… [+192 chars]'}, {'source': {'id': None, 'name': 'Zdnet.co.kr'}, 'author': '손희연 기자', 'title': '[미장브리핑] 사우디와 6천억달러 규모 투자…엔비디아·AMD 상승', 'description': '[지디넷코리아]◇ 13일(현지시간) 미국 증시▲다우존스산업평균(다우)지수 전 거래일 대비 0.64% 하락한 42140.43.▲스탠다드앤푸어스(S&P)500 지수 전 

{'금리': {'topic': '금리',
  'average_sentiment': 0.0,
  'analyzed_articles': 6,
  'sentiment_summary': '중립적'},
 '통화정책': {'topic': '통화정책',
  'average_sentiment': 0.8,
  'analyzed_articles': 2,
  'sentiment_summary': '긍정적'},
 '인플레이션': {'topic': '인플레이션',
  'average_sentiment': -0.32,
  'analyzed_articles': 4,
  'sentiment_summary': '부정적'},
 'GDP': {'topic': 'GDP',
  'average_sentiment': -0.0,
  'analyzed_articles': 10,
  'sentiment_summary': '중립적'}}

In [63]:
# 산업군 관련 뉴스기사 감성 분석 도구
@tool
def analyze_industry_news(industry: str) -> Dict[str, Any]:
    """특정 산업군에 대한 뉴스를 분석합니다."""
    return get_news_sentiment(f"{industry}")

In [64]:
analyze_industry_news("한화")

[{'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '이두리 기자 redo@kyunghyang.com', 'title': '12회 연속 ‘히트’…한화 드라마 ‘대박’', 'description': '와이스, 8이닝 무실점 ‘철벽’\n키움 대파…연승 기록 추가\n“5명 선발 투수들 서로 지원”\n전신 빙그레 14연승에 ‘성큼’\n\n한화 라이언 와이스가 11일 고척 스카이돔에서 열린 키움전에서 깔끔하게 이닝을 마친 뒤 미소 지으며 마운드를 내려오고 있다. 연합뉴스\n\n한화가 지는 법을 잊었다. 벌써 12연승이다. 구단 최다 연승 기록인 14연승까지 2···', 'url': 'https://www.khan.co.kr/article/202505112020015', 'urlToImage': 'https://img.khan.co.kr/news/2025/05/11/l_2025051201000278400025171.jpg', 'publishedAt': '2025-05-11T11:20:01Z', 'content': '12 \r\n11 . \r\n. 12. 14 2 .\r\n 11 8-0 . 33 11 .\r\n 12 1992 523 . 2 14 . .\r\n . 1~2 4 3 4 .\r\n 3 . 5-0 9 , 3 .\r\n 6 . 8 93 1 2, 9 .\r\n 8 , , , 93 1 .\r\n . 5 .\r\n 6 1 . 16 55 3.73 8 6 .\r\n .\r\n . 1 .\r\n . 11 . 2 2 .… [+21 chars]'}, {'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '이성희 기자 mong2@kyunghyang.com', 'title': '‘새우가 고래 삼켰다’ 한화, 아워홈 인수···단체급식 판도 바뀌나', 'description': '‘한화 삼남’ 김동선 부사장 주도\n업계선 ‘범LG가 물량’ 어디로 갈까 관심\n구지은 부회장 향후 반발 등 과

{'topic': '한화',
 'average_sentiment': 0.48,
 'analyzed_articles': 10,
 'sentiment_summary': '긍정적'}

In [59]:
# 투자 결정 도구
@tool
def make_investment_decision(inputs: Dict[str, Any]) -> Dict[str, Any]:
    """
    종합적인 분석을 바탕으로 투자 결정을 제안합니다.
    """

    # 입력 값 추출
    company = inputs.get("company", "Unknown")
    company_sentiment = inputs.get("company_sentiment", {"average_sentiment": 0})
    industry_sentiment = inputs.get("industry_sentiment", {"average_sentiment": 0})
    economic_indicators = inputs.get("economic_indicators", {})

    # 가중치 설정
    weights = {
        "company": 0.4,
        "industry": 0.3,
        "economic": 0.3
    }

    # 회사 점수 계산
    company_score = company_sentiment.get("average_sentiment", 0) * weights["company"]

    # 산업 점수 계산
    industry_score = industry_sentiment.get("average_sentiment", 0) * weights["industry"]

    # 경제 지표 점수 계산
    economic_scores = [
        indicator.get("average_sentiment", 0)
        for indicator in economic_indicators.values()
    ]
    economic_score = (sum(economic_scores) / len(economic_scores)) * weights["economic"] if economic_scores else 0

    # 총점 계산
    total_score = company_score + industry_score + economic_score

    # 투자 결정
    decision = {
        "company": company,
        "timestamp": datetime.now().isoformat(),
        "total_score": round(total_score, 2),
        "recommendation": "매수" if total_score > 0.3 else "매도" if total_score < -0.3 else "관망",
        "confidence": abs(total_score),
        "factors": {
            "company_sentiment": company_sentiment,
            "industry_sentiment": industry_sentiment,
            "economic_indicators": economic_indicators
        }
    }

    return decision

In [71]:
class InvestmentAnalyzer:
    def __init__(self):
        self.llm = ChatOpenAI(
            model="gpt-4",
            temperature=0
        )

        self.tools = [
            get_news_sentiment,
            analyze_industry_news,
            analyze_economic_indicators,
            make_investment_decision
        ]

        prompt = ChatPromptTemplate.from_messages([
            ("system", """주식 시장 분석 및 투자 결정을 돕는 AI 에이전트입니다.
            뉴스 데이터를 수집하고 감성 분석을 통해 투자 결정을 제안합니다."""),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        self.agent = create_openai_functions_agent(self.llm, self.tools, prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True)

    def analyze(self, company: str, industry: str) -> Dict[str, Any]:
        """투자 분석을 수행하고 결과를 반환합니다."""
        try:
            # 1. 회사 감성 분석
            company_result = get_news_sentiment(company)

            # 2. 산업군 감성 분석
            industry_result = analyze_industry_news(industry)

            # 3. 경제 지표 분석
            economic_result = analyze_economic_indicators({"list": ["금리"]})

            # 4. 투자 결정
            decision = make_investment_decision({"inputs":{
                    "company": company,
                    "company_sentiment": company_result,
                    "industry_sentiment": industry_result,
                    "economic_indicators": economic_result
                }
            })
            # decision = self.executor.invoke({
            #     "tool": "make_investment_decision",
            #     "input": {
            #         "company": company,
            #         "company_sentiment": company_result,
            #         "industry_sentiment": industry_result,
            #         "economic_indicators": economic_result
            #     }
            # })

            return decision

        except Exception as e:
            print(f"분석 중 오류 발생: {str(e)}")
            return {
                "error": str(e),
                "status": "failed"
            }

In [74]:
analyzer = InvestmentAnalyzer()

# 분석 예시
result = analyzer.analyze(
    company="한화",
    industry="방산"
)

print("\n최종 투자 분석 결과:")
print(json.dumps(result, indent=2, ensure_ascii=False))

[{'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '이두리 기자 redo@kyunghyang.com', 'title': '12회 연속 ‘히트’…한화 드라마 ‘대박’', 'description': '와이스, 8이닝 무실점 ‘철벽’\n키움 대파…연승 기록 추가\n“5명 선발 투수들 서로 지원”\n전신 빙그레 14연승에 ‘성큼’\n\n한화 라이언 와이스가 11일 고척 스카이돔에서 열린 키움전에서 깔끔하게 이닝을 마친 뒤 미소 지으며 마운드를 내려오고 있다. 연합뉴스\n\n한화가 지는 법을 잊었다. 벌써 12연승이다. 구단 최다 연승 기록인 14연승까지 2···', 'url': 'https://www.khan.co.kr/article/202505112020015', 'urlToImage': 'https://img.khan.co.kr/news/2025/05/11/l_2025051201000278400025171.jpg', 'publishedAt': '2025-05-11T11:20:01Z', 'content': '12 \r\n11 . \r\n. 12. 14 2 .\r\n 11 8-0 . 33 11 .\r\n 12 1992 523 . 2 14 . .\r\n . 1~2 4 3 4 .\r\n 3 . 5-0 9 , 3 .\r\n 6 . 8 93 1 2, 9 .\r\n 8 , , , 93 1 .\r\n . 5 .\r\n 6 1 . 16 55 3.73 8 6 .\r\n .\r\n . 1 .\r\n . 11 . 2 2 .… [+21 chars]'}, {'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '이성희 기자 mong2@kyunghyang.com', 'title': '‘새우가 고래 삼켰다’ 한화, 아워홈 인수···단체급식 판도 바뀌나', 'description': '‘한화 삼남’ 김동선 부사장 주도\n업계선 ‘범LG가 물량’ 어디로 갈까 관심\n구지은 부회장 향후 반발 등 과

단순 금리에 대한 뉴스만 불러올 경우, 아래 코드 활용 가능

In [75]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, initialize_agent, Tool
import requests
import json

@tool()
def get_news_sentiment(topic:str) -> str:
    """주제와 관련된 뉴스 기사의 감성어를 가져와 분석"""
    news_api_url = "https://newsapi.org/v2/everything"
    querystring = {"q": topic, "apiKey": os.getenv('NEWS_API_KEY')}

    response = requests.get(news_api_url, params=querystring)
    articles = response.json().get('articles', [])
    print(articles)

    # 간단한 감정 분석 예시
    sentiment_score = sum(1 if "positive" in article['description'].lower() else -1 for article in articles)
    return f"Sentiment score for {topic}: {sentiment_score}"

tools = [get_news_sentiment]

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", """주식 시장 분석 및 투자 결정을 돕는 AI 에이전트입니다.
    뉴스 데이터를 수집하고 감성 분석을 통해 투자 결정을 제안합니다."""),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_openai_functions_agent(llm, tools, prompt)


agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 주가 예측 시나리오
response = agent_executor.invoke({"input": "오늘 금리에 대해 감정 분석을 알려줘"})
result = response['output']
print(result)



> Entering new AgentExecutor chain...

Invoking: `get_news_sentiment` with `{'topic': '금리'}`


[{'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': 'KyungHyangSinmun', 'title': '[사설] 관세·내수 비상에도 금리 동결, 한은 ‘역성장’ 경고했다', 'description': '경기 부양엔 기준금리 인하가 즉효약이지만 한국은행은 17일 기준금리를 연 2.75%로 동결했다. 극심한 경기 침체에도 금리 인하 카드를 쓸 수 없을 정도로 경제가 총체적 난국이라는 의미다. 이창용 한은 총재는 “전망의 기본 시나리오조차 설정하기 어렵다”고 말했다.\n\n금리를 낮추면 당장 원·달러 환율 상승이 두렵다. 그러잖아도 ‘트럼프’와 ‘윤석열’ 변수로 ···', 'url': 'https://www.khan.co.kr/article/202504171815001', 'urlToImage': 'https://img.khan.co.kr/news/2025/04/17/news-p.v1.20250417.ed17a295602d484a8895f91447c928a9_P1.jpeg', 'publishedAt': '2025-04-17T09:15:00Z', 'content': '17 2.75% . . .\r\n · . . 1.75% . 1998 2009 .\r\n . 1 , 2 . 4 . . (WTO) · (GDP) 7% . .\r\n . , 40 . , . 15~20 · 12 . , .\r\n , . () . 11 . .\r\n17 .'}, {'source': {'id': None, 'name': 'Khan.co.kr'}, 'author': '워싱턴 | 김유진 특파원 yjkim@kyunghyang.com', 'title': '트럼프, 연준 의장에 “실패자”라며 금리 인하 압박', 'description': '도널드 트럼프 미국 대통령이 21일(현지시간)